In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

# Hugging Face & LoRA
from transformers import AutoImageProcessor, AutoModelForDepthEstimation
from peft import LoraConfig, get_peft_model

# ==========================================
# 1. CONFIGURATION
# ==========================================
FORCE_CPU = True 

MODEL_ID = "depth-anything/Depth-Anything-V2-Small-hf"
OUTPUT_DIR = "./resultats_projet_final"
BATCH_SIZE = 4
LR = 1e-4
EPOCHS = 40 
PATIENCE = 4 

if FORCE_CPU:
    DEVICE = "cpu"
    print("⚠️ MODE CPU FORCÉ")
else:
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"🚀 Démarrage sur : {DEVICE}")

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ==========================================
# 2. CLASS CALLBACK 
# ==========================================
class EarlyStopping:
    """
    Arrête l'entraînement si le score ne s'améliore pas après un certain nombre d'époques.
    """
    def __init__(self, patience=3, verbose=False, delta=0, path='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.inf 
        self.delta = delta
        self.path = os.path.join(OUTPUT_DIR, "meilleur_modele_lora")
        self.best_epoch = 0

    def __call__(self, score, model, epoch):
        # On maximise le Delta (plus c'est haut, mieux c'est)
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(score, model)
            self.best_epoch = epoch
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'   ⏳ Pas d\'amélioration ({self.counter}/{self.patience})')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(score, model)
            self.counter = 0
            self.best_epoch = epoch

    def save_checkpoint(self, score, model):
        if self.verbose:
            print(f'   🔥 Nouveau record ! (Delta: {score:.4f}). Sauvegarde...')
        model.save_pretrained(self.path)

# ==========================================
# 3. DATASET
# ==========================================
class ZividDataset(Dataset):
    def __init__(self, root_dir, processor):
        self.img_dir = os.path.join(root_dir, "images")
        self.depth_dir = os.path.join(root_dir, "depth")
        self.processor = processor
        valid_ext = ('.png', '.jpg', '.jpeg')
        if not os.path.exists(self.img_dir): raise FileNotFoundError(f"❌ Dossier introuvable")
        self.images = sorted([f for f in os.listdir(self.img_dir) if f.lower().endswith(valid_ext)])

    def __len__(self): return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        base_name = os.path.splitext(img_name)[0].replace("_color", "")
        npy_name = base_name + "_rawDepth.npy"
        img_path = os.path.join(self.img_dir, img_name)
        npy_path = os.path.join(self.depth_dir, npy_name)

        image = Image.open(img_path).convert("RGB")
        try:
            point_cloud = np.load(npy_path)
            depth_Z = point_cloud[:, :, 2]
            if np.nanmax(depth_Z) > 100: depth_Z = depth_Z / 1000.0
        except: return None

        inputs = self.processor(images=image, return_tensors="pt")
        target_h, target_w = inputs["pixel_values"].shape[-2:]
        depth_tensor = torch.from_numpy(depth_Z).float().unsqueeze(0).unsqueeze(0)
        mask = ~torch.isnan(depth_tensor) & ~torch.isinf(depth_tensor) & (depth_tensor > 0)
        depth_tensor = torch.nan_to_num(depth_tensor, nan=0.0)
        
        depth_resized = torch.nn.functional.interpolate(depth_tensor, size=(target_h, target_w), mode='nearest')
        mask_resized = torch.nn.functional.interpolate(mask.float(), size=(target_h, target_w), mode='nearest')
        
        return {"pixel_values": inputs["pixel_values"].squeeze(0), "labels": depth_resized.squeeze(0), "mask": mask_resized.squeeze(0)}

# ==========================================
# 4. FONCTIONS UTILITAIRES
# ==========================================
def compute_metrics(pred, target, mask):
    pred = pred[mask]; target = target[mask]
    if len(target) == 0: return 0.0, 0.0, 0.0
    abs_rel = torch.mean(torch.abs(pred - target) / target)
    rmse = torch.sqrt(torch.mean((pred - target) ** 2))
    max_ratio = torch.max(pred / target, target / pred)
    delta1 = (max_ratio < 1.25).float().mean()
    return abs_rel.item(), rmse.item(), delta1.item()

def save_sample_image(pixel_values, true_depth, pred_depth, epoch):
    img = pixel_values.permute(1, 2, 0).cpu().numpy()
    img = (img - img.min()) / (img.max() - img.min())
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1); plt.imshow(img); plt.title("Input RGB"); plt.axis('off')
    plt.subplot(1, 3, 2); plt.imshow(true_depth.squeeze().cpu().numpy(), cmap='inferno'); plt.title("Vérité"); plt.axis('off')
    plt.subplot(1, 3, 3); plt.imshow(pred_depth.squeeze().detach().cpu().numpy(), cmap='inferno'); plt.title(f"Pred (Ep {epoch})"); plt.axis('off')
    plt.savefig(os.path.join(OUTPUT_DIR, f"visu_epoch_{epoch}.png")); plt.close()

def save_curves(history):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1); plt.plot(history['loss'], label='Loss'); plt.legend(); plt.grid()
    plt.subplot(1, 2, 2); plt.plot(history['delta'], label='Delta'); plt.legend(); plt.grid()
    plt.savefig(os.path.join(OUTPUT_DIR, "courbes.png")); plt.close()

# ==========================================
# 5. MAIN
# ==========================================
def run_project():
    base_dir = os.getcwd()
    dataset_dir = os.path.join(base_dir, "DATASET_DEVOIR")
    
    print("⏳ Chargement...")
    processor = AutoImageProcessor.from_pretrained(MODEL_ID)
    model = AutoModelForDepthEstimation.from_pretrained(MODEL_ID)
    
    # --- CONFIG LORA AMÉLIORÉE ---
    lora_config = LoraConfig(
        r=32, lora_alpha=32, 
        target_modules=["query", "key", "value", "dense"], 
        lora_dropout=0.1, bias="none"
    )
    model = get_peft_model(model, lora_config)
    model.to(DEVICE)
    model.print_trainable_parameters()
    
    try:
        dataset = ZividDataset(dataset_dir, processor)
        dataset = [d for d in dataset if d is not None]
    except Exception as e: print(e); return
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    history = {'loss': [], 'delta': []}
    
    # --- INITIALISATION DU CALLBACK ---
    early_stopping = EarlyStopping(patience=PATIENCE, verbose=True)

    print(f"\n🔥 Démarrage (Max Epochs: {EPOCHS}, Patience: {PATIENCE})")
    
    for epoch in range(1, EPOCHS + 1):
        total_loss, total_delta = 0, 0
        model.train()
        
        loop = tqdm(loader, desc=f"Ep {epoch}/{EPOCHS}")
        for batch_idx, batch in enumerate(loop):
            pixel_values = batch["pixel_values"].to(DEVICE)
            labels = batch["labels"].to(DEVICE)
            mask = batch["mask"].to(DEVICE)
            
            optimizer.zero_grad()
            outputs = model(pixel_values=pixel_values)
            prediction = torch.nn.functional.interpolate(
                outputs.predicted_depth.unsqueeze(1), size=labels.shape[-2:], mode="bilinear", align_corners=False
            )
            
            loss = torch.sum(mask * (prediction - labels)**2) / (torch.sum(mask) + 1e-6)
            loss.backward()
            optimizer.step()
            
            with torch.no_grad():
                _, _, d1 = compute_metrics(prediction, labels, mask.bool())
            
            total_loss += loss.item()
            total_delta += d1
            loop.set_postfix(Loss=f"{loss.item():.3f}", Delta=f"{d1:.3f}")

            if batch_idx == 0: save_sample_image(pixel_values[0], labels[0], prediction[0], epoch)

        avg_loss = total_loss / len(loader)
        avg_delta = total_delta / len(loader)
        history['loss'].append(avg_loss)
        history['delta'].append(avg_delta)
        
        print(f"🏁 Epoch {epoch} | Delta Moy: {avg_delta:.4f}")

        # --- APPEL DU CALLBACK ---
        # On lui donne le score Delta actuel et le modèle
        early_stopping(avg_delta, model, epoch)
        
        if early_stopping.early_stop:
            print("\n🛑 Early Stopping déclenché ! Le modèle ne progresse plus.")
            print(f"Meilleure performance atteinte à l'époque {early_stopping.best_epoch} (Delta: {early_stopping.best_score:.4f})")
            break

    save_curves(history)
    print(f"\n✅ Terminé. Le meilleur modèle est sauvegardé ici : {early_stopping.path}")

if __name__ == "__main__":
    run_project()

⚠️ MODE CPU FORCÉ
⏳ Chargement...
trainable params: 1,179,648 || all params: 25,964,737 || trainable%: 4.5433

🔥 Démarrage (Max Epochs: 40, Patience: 4)


Ep 1/40: 100%|██████████| 15/15 [01:38<00:00,  6.57s/it, Delta=0.454, Loss=0.527]


🏁 Epoch 1 | Delta Moy: 0.2298
   🔥 Nouveau record ! (Delta: 0.2298). Sauvegarde...


Ep 2/40: 100%|██████████| 15/15 [01:42<00:00,  6.82s/it, Delta=0.462, Loss=0.296]


🏁 Epoch 2 | Delta Moy: 0.4229
   🔥 Nouveau record ! (Delta: 0.4229). Sauvegarde...


...

🏁 Epoch 36 | Delta Moy: 0.9858
   ⏳ Pas d'amélioration (2/4)


Ep 37/40: 100%|██████████| 15/15 [01:34<00:00,  6.28s/it, Delta=0.984, Loss=0.009]


🏁 Epoch 37 | Delta Moy: 0.9856
   ⏳ Pas d'amélioration (3/4)


Ep 38/40: 100%|██████████| 15/15 [01:35<00:00,  6.36s/it, Delta=0.973, Loss=0.023]


🏁 Epoch 38 | Delta Moy: 0.9784
   ⏳ Pas d'amélioration (4/4)

🛑 Early Stopping déclenché ! Le modèle ne progresse plus.
Meilleure performance atteinte à l'époque 34 (Delta: 0.9861)

✅ Terminé. Le meilleur modèle est sauvegardé ici : ./resultats_projet_final\meilleur_modele_lora
